# Modelling

In [1]:
import pandas as pd
import numpy as np

from scipy import sparse

from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split

from resources import *

#Import Warnings
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
%run resources.py

## Load data

In [3]:
# Load user items data
recdata = pd.read_csv('../Data/steam/recdata.csv', index_col=0)
recdata = recdata.rename(columns = {'variable':'id', 'value': 'owned'})
recdata.head()

,id,uid,owned
0,10,0,1.0
1,10,1,1.0
2,10,3,1.0
3,10,4,1.0
4,10,10,1.0


In [4]:
# Load game names data
games = pd.read_csv('../Data/steam/gamenames.csv', index_col = 0)
games.head()

,title,id
0,Lost Summoner Kitty,761140.0
1,Ironbound,643980.0
2,Real Pool 3D - Poolians,670290.0
3,弹炸人2222,767400.0
4,NaN,773570.0


## Additional Preprocessing

### Create interaction matrix

In [12]:
# Use create_interaction_matrix function
interactions = create_interaction_matrix(df = recdata,
                                         user_col = 'uid',
                                         item_col = 'id',
                                         rating_col = 'owned')
interactions.shape

(69277, 8791)

In [6]:
interactions.head()

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
type(interactions)

pandas.core.frame.DataFrame

### Train test split

In [41]:
# Get number of users
len(interactions)

69277

In [42]:
# Establish number of users in train/test sets

train_num = round((80/100)*len(interactions),0)
print(f'We desire {train_num} users in our training set.')

test_num = len(interactions)-train_num
print(f'We desire {test_num} users in our test set.')

We desire 55422.0 users in our training set.
We desire 13855.0 users in our test set.


In [43]:
# Define train and test sets
train = interactions[:55422]
test = interactions[55422:]

### Create user dictionary

In [44]:
# Create user dictionary using helper function
user_dict = create_user_dict(interactions=interactions)

### Create item dictionary

In [45]:
# Create game dictionary using helper function
games_dict = create_item_dict(df = games, id_col = 'id', name_col = 'title')

### Create sparse matrices

In [46]:
# Create sparse matrices for evaluation 
train_sparse = sparse.csr_matrix(train.values)

#Add X users to Test so that the number of rows in Train match Test
N = train.shape[0] #Rows in Train set
n,m = test.shape #Rows & columns in Test set
z = np.zeros([(N-n),m]) #Create the necessary rows of zeros with m columns
#test = test.toarray() #Temporarily convert Test into a numpy array
test = np.vstack((test,z)) #Vertically stack Test on top of the blank users
test_sparse = sparse.csr_matrix(test) #Convert back to sparse

## Modelling using LightFM

### WARP loss model

In [47]:
# Instantiate and fit model
mf_model = run_model(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [48]:
# Get precision
train_precision = precision_at_k(mf_model, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.66, test 0.19.


In [49]:
# Get AUC
train_auc = auc_score(mf_model, train_sparse).mean()
test_auc = auc_score(mf_model, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.99, test 0.94.


### BPR loss model

In [50]:
# Instantiate and fit model
mf_model_bpr = run_model(interactions = train,
                 n_components = 30,
                 loss = 'bpr',
                 epoch = 30,
                 n_jobs = 4)

In [51]:
# Get precision
train_precision = precision_at_k(mf_model_bpr, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model_bpr, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.75, test 0.15.


In [52]:
# Get AUC
train_auc = auc_score(mf_model_bpr, train_sparse).mean()
test_auc = auc_score(mf_model_bpr, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.97, test 0.73.


### Adjust component number

### Final model

## User Recommendations

### Recommendations for existing user

In [58]:
# Get recommendations
rec_list = get_recs(model = mf_model, 
                    interactions = interactions, 
                    user_id = 12, 
                    user_dict = user_dict,
                    item_dict = games_dict, 
                    threshold = 0,
                    num_items = 5,
                    show_known = True, 
                    show_recs = True)

Known Likes:
1- Epic Battle Fantasy 3
2- Paladins®
3- Shakes and Fidget
4- Tactical Craft Online
5- Crusaders of the Lost Idols
6- Time Clickers
7- Dragon Saga
8- Blender
9- Clicker Heroes
10- Spooky's Jump Scare Mansion
11- Codename CURE
12- Fire With Fire Tower Attack and Defense
13- Modular Combat
14- AdVenture Capitalist
15- BrainBread 2
16- Brick-Force
17- Dirty Bomb®
18- Five Nights at Freddy's 2
19- TERA
20- Don't Starve Together
21- Double Action: Boogaloo
22- The Way of Life Free Edition
23- Unturned
24- Trove
25- BLOCKADE 3D
26- Robocraft
27- Block N Load
28- Brawlhalla
29- Warface
30- Quake Live™
31- Lambda Wars Beta
32- Fistful of Frags
33- GunZ 2: The Second Duel
34- BattleBlock Theater®
35- Dungeon Defenders II
36- Warframe
37- Sven Co-op
38- Defiance
39- No More Room in Hell
40- PlanetSide 2
41- Gear Up
42- The Lord of the Rings Online™
43- Blacklight: Retribution
44- Gotham City Impostors Free to Play
45- Everquest ®
46- Castle Crashers®
47- Orcs Must Die! 2
48- Realm o

In [ ]:
### Add comments - Expect additional CS game etc

### Recommendations for new user

In [8]:
# Get list of owned games from user

# Create empty list to store game ids
ownedgames = []

# Create loop to prompt for game id and ask if want to continue
moretoadd = 'y'
while moretoadd == 'y':
    game = input('Please enter the game id: ')
    ownedgames.append(game)
    moretoadd = input('Do you have more games to add? y/n ')
    
# Print list of owned games
print(f"You own the following games: {ownedgames}")

Please enter the game id: 8
Do you have more games to add? y/n n
You own the following games: ['8']


In [15]:
# View last entry in interactions matrix
interactions[-1:]

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
88308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
 # Get new user index
newuidindex = interactions.index[-1]+1
newuidindex

88309

In [17]:
# Add row for new user id with zeros
interactions.loc[newuidindex] = 0
interactions

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Change value to 1 where user owns the game

for gameid in ownedgames:
    interactions.loc[newuidindex, gameid] = 1

In [ ]:
# Recreate dictionaries
user_dict = create_user_dict(interactions=interactions)
games_dict = create_item_dict(df = games, id_col = 'id', name_col = 'title')

In [ ]:
# Fit model
mf_model = run_model(interactions = train,
                     n_components = 30,
                     loss = 'warp',
                     epoch = 30,
                     n_jobs = 4)

In [ ]:
# Get recommendations
rec_list = get_recs(model = mf_model, 
                    interactions = interactions, 
                    user_id = newuidindex, 
                    user_dict = user_dict,
                    item_dict = games_dict, 
                    threshold = 0,
                    num_items = 5,
                    show_known = True, 
                    show_recs = True)

## Item Recommendations

### Create item embedding matrix

In [64]:
item_dist = create_item_emdedding_matrix(model = mf_model,interactions = interactions)

In [66]:
item_dist.shape

(8791, 8791)

In [67]:
item_dist.head()

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
id,,,,,,,,,,,,,,,,,,,,,
10,1.000000,0.703092,0.867419,0.871559,0.684434,0.869805,0.683555,0.993782,0.683072,0.510813,...,0.041585,0.070690,-0.076905,-0.086024,0.070537,0.008223,-0.148230,-0.115156,0.014385,-0.056317
20,0.703092,1.000000,0.845572,0.843557,0.997379,0.839401,0.986706,0.663473,0.997265,0.847267,...,-0.166892,-0.104024,-0.078415,-0.054945,-0.042555,-0.072952,-0.243327,-0.226339,-0.004695,-0.075152
30,0.867419,0.845572,1.000000,0.996962,0.839656,0.996300,0.812246,0.819267,0.840940,0.617789,...,0.057061,-0.125987,0.019689,-0.035818,0.096059,0.076818,-0.079370,-0.075650,0.087320,0.050368
40,0.871559,0.843557,0.996962,1.000000,0.838675,0.999104,0.810467,0.822601,0.839866,0.608376,...,0.067647,-0.125478,0.008246,-0.021051,0.112209,0.093214,-0.075856,-0.070456,0.091169,0.053585
50,0.684434,0.997379,0.839656,0.838675,1.000000,0.834340,0.989329,0.641949,0.999489,0.859258,...,-0.184936,-0.122161,-0.089101,-0.062516,-0.046453,-0.075597,-0.248418,-0.238196,-0.020810,-0.079319


### Generate item recommendations

In [70]:
# Similar items to item_id 10 - Counter Strike
item_rec_list = get_item_recs(item_emdedding_matrix = item_dist, 
                              item_id = 10, 
                              item_dict = games_dict, 
                              n_items = 10, 
                              show = True)

Item of interest: Counter-Strike
Similar items:
1- Counter-Strike: Condition Zero
2- Deathmatch Classic
3- Ricochet
4- Day of Defeat
5- Counter-Strike: Source
6- Team Fortress Classic
7- Day of Defeat: Source
8- Half-Life: Opposing Force
9- Half-Life
10- Half-Life: Blue Shift


In [71]:
games[games['title'] == 'The Witness']

,title,id
5211,The Witness,210970.0


In [72]:
# Similar items to item_id 210970 The Witness
item_rec_list = get_item_recs(item_emdedding_matrix = item_dist, 
                              item_id = 210970, 
                              item_dict = games_dict, 
                              n_items = 10, 
                              show = True)

Item of interest: The Witness
Similar items:
1- Headlander
2- ABZU
3- SUPERHOT
4- Assault Android Cactus
5- Oxenfree
6- Quadrilateral Cowboy
7- Poly Bridge
8- The Guest
9- Firewatch
10- Grow Up


In [78]:
gamesdata = pd.read_csv('gamesdata.csv', index_col = 0)
gamesdata[gamesdata['id'].isin(item_rec_list)]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
1550,Witch Beam,"['Action', 'Indie']",Assault Android Cactus,Assault Android Cactus,http://store.steampowered.com/app/250110/Assau...,2015-09-23,"['Twin Stick Shooter', 'Indie', 'Action', 'Bul...",NaN,http://steamcommunity.com/app/250110/reviews/?...,"['Single-player', 'Shared/Split Screen', 'Stea...",14.99,False,250110.0,Witch Beam,Overwhelmingly Positive,79.0
5143,Night School Studio,"['Adventure', 'Indie']",Oxenfree,Oxenfree,http://store.steampowered.com/app/388880/Oxenf...,2016-01-14,"['Story Rich', 'Choices Matter', 'Supernatural...",NaN,http://steamcommunity.com/app/388880/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,388880.0,Night School Studio,Very Positive,80.0
5500,SUPERHOT Team,"['Action', 'Indie']",SUPERHOT,SUPERHOT,http://store.steampowered.com/app/322500/SUPER...,2016-02-25,"['FPS', 'Action', 'Bullet Time', 'First-Person...",NaN,http://steamcommunity.com/app/322500/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",24.99,False,322500.0,SUPERHOT Team,Very Positive,82.0
5513,505 Games,"['Adventure', 'Indie']",The Guest,The Guest,http://store.steampowered.com/app/402040/The_G...,2016-03-10,"['Indie', 'Adventure', 'Puzzle', 'Atmospheric'...",NaN,http://steamcommunity.com/app/402040/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",9.99,False,402040.0,Team Gotham,Mostly Positive,68.0
6783,Adult Swim Games,['Action'],Headlander,Headlander,http://store.steampowered.com/app/340000/Headl...,2016-07-25,"['Action', 'Sci-fi', 'Metroidvania', 'Platform...",NaN,http://steamcommunity.com/app/340000/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,340000.0,Double Fine Productions,Very Positive,74.0
6976,Ubisoft,"['Adventure', 'Casual', 'Indie']",Grow Up,Grow Up,http://store.steampowered.com/app/426790/Grow_Up/,2016-08-16,"['Adventure', 'Casual', 'Indie', 'Open World',...",NaN,http://steamcommunity.com/app/426790/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",9.99,False,426790.0,"Reflections, a Ubisoft Studio",Very Positive,71.0
22123,505 Games,"['Action', 'Adventure', 'Casual', 'Indie', 'Si...",ABZU,ABZU,http://store.steampowered.com/app/384190/ABZU/,2016-08-02,"['Relaxing', 'Atmospheric', 'Underwater', 'Gre...",NaN,http://steamcommunity.com/app/384190/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,384190.0,Giant Squid,Very Positive,83.0
22244,Blendo Games,"['Action', 'Adventure', 'Indie']",Quadrilateral Cowboy,Quadrilateral Cowboy,http://store.steampowered.com/app/240440/Quadr...,2016-07-25,"['Indie', 'Adventure', 'Action', 'Hacking', 'C...",NaN,http://steamcommunity.com/app/240440/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",19.99,False,240440.0,Blendo Games,Very Positive,81.0
22364,Dry Cactus,"['Indie', 'Simulation']",Poly Bridge,Poly Bridge,http://store.steampowered.com/app/367450/Poly_...,2016-07-12,"['Building', 'Simulation', 'Physics', 'Puzzle'...",NaN,http://steamcommunity.com/app/367450/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",11.99,False,367450.0,Dry Cactus,Very Positive,73.0
24216,"Panic Inc., Campo Santo","['Adventure', 'Indie']",Firewatch,Firewatch,http://store.steampowered.com/app/383870/Firew...,2016-02-09,"['Atmospheric', 'Adventure', 'Walking Simulato...",NaN,http://steamcommunity.com/app/383870/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,383870.0,Campo Santo,Very Positive,81.0
